# Telco Customer Churn Analysis - Complete EDA & Visualization
## Data Cleaning, Exploratory Data Analysis, and Advanced Visualizations

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

## 1. Data Loading and Initial Exploration

In [ ]:
# Load dataset
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nData Types:")
print(df.dtypes)
print(f"\nFirst 5 rows:")
df.head()

## 2. Data Cleaning

In [ ]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum())

# Check for empty strings or spaces
print("\nEmpty/Space Values:")
for col in df.columns:
    empty_count = (df[col] == '').sum() + (df[col] == ' ').sum()
    if empty_count > 0:
        print(f"{col}: {empty_count}")

# Fix TotalCharges column (convert to numeric)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
print(f"\nTotalCharges missing after conversion: {df['TotalCharges'].isnull().sum()}")

# Fill missing TotalCharges with 0 (new customers)
df['TotalCharges'].fillna(0, inplace=True)

# Create binary target variable
df['Churn_Binary'] = df['Churn'].map({'Yes': 1, 'No': 0})

print(f"\nCleaned dataset shape: {df.shape}")
print(f"Churn distribution: {df['Churn'].value_counts()}")

## 3. Basic Statistical Analysis

In [ ]:
# Numerical columns summary
numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
print("Numerical Columns Summary:")
print(df[numerical_cols].describe())

# Categorical columns summary
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('customerID')  # Remove ID column

print(f"\nCategorical Columns: {len(categorical_cols)}")
for col in categorical_cols[:5]:  # Show first 5
    print(f"\n{col}: {df[col].value_counts().to_dict()}")

## 4. Advanced Visualizations - Basic Charts

In [ ]:
# 1. Churn Distribution (Pie Chart)
fig = make_subplots(rows=1, cols=2, 
                    specs=[[{"type": "pie"}, {"type": "bar"}]],
                    subplot_titles=("Churn Distribution", "Churn by Gender"))

# Pie chart
churn_counts = df['Churn'].value_counts()
fig.add_trace(go.Pie(labels=churn_counts.index, values=churn_counts.values,
                     colors=['#ff6b6b', '#4ecdc4']), row=1, col=1)

# Bar chart by gender
gender_churn = df.groupby(['gender', 'Churn']).size().unstack()
fig.add_trace(go.Bar(name='No Churn', x=gender_churn.index, y=gender_churn['No'],
                     marker_color='#4ecdc4'), row=1, col=2)
fig.add_trace(go.Bar(name='Churn', x=gender_churn.index, y=gender_churn['Yes'],
                     marker_color='#ff6b6b'), row=1, col=2)

fig.update_layout(height=400, title_text="Customer Churn Analysis")
fig.show()

In [ ]:
# 2. Contract Type Analysis (Stacked Bar Chart)
contract_churn = pd.crosstab(df['Contract'], df['Churn'], normalize='index') * 100

fig = go.Figure()
fig.add_trace(go.Bar(name='No Churn', x=contract_churn.index, y=contract_churn['No'],
                     marker_color='#4ecdc4'))
fig.add_trace(go.Bar(name='Churn', x=contract_churn.index, y=contract_churn['Yes'],
                     marker_color='#ff6b6b'))

fig.update_layout(barmode='stack', title='Churn Rate by Contract Type (%)',
                  xaxis_title='Contract Type', yaxis_title='Percentage')
fig.show()

print("Churn Rate by Contract:")
print(df.groupby('Contract')['Churn_Binary'].mean() * 100)

## 5. Advanced Visualizations - Distribution Analysis

In [ ]:
# 3. Tenure Distribution by Churn (Histogram + Box Plot)
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Tenure Distribution by Churn", "Tenure Box Plot by Churn"))

# Histogram
for churn_status in ['No', 'Yes']:
    data = df[df['Churn'] == churn_status]['tenure']
    fig.add_trace(go.Histogram(x=data, name=f'Churn: {churn_status}',
                               opacity=0.7, nbinsx=30), row=1, col=1)

# Box plot
for churn_status in ['No', 'Yes']:
    data = df[df['Churn'] == churn_status]['tenure']
    fig.add_trace(go.Box(y=data, name=f'Churn: {churn_status}'), row=2, col=1)

fig.update_layout(height=600, title_text="Tenure Analysis")
fig.show()

print(f"Average tenure - No Churn: {df[df['Churn']=='No']['tenure'].mean():.1f} months")
print(f"Average tenure - Churn: {df[df['Churn']=='Yes']['tenure'].mean():.1f} months")

In [ ]:
# 4. Monthly Charges vs Total Charges (Scatter Plot)
fig = px.scatter(df, x='MonthlyCharges', y='TotalCharges', color='Churn',
                 title='Monthly Charges vs Total Charges by Churn Status',
                 hover_data=['tenure', 'Contract'],
                 color_discrete_map={'No': '#4ecdc4', 'Yes': '#ff6b6b'})

fig.update_layout(height=500)
fig.show()

# Correlation analysis
print("Correlation with Churn:")
correlations = df[['tenure', 'MonthlyCharges', 'TotalCharges', 'Churn_Binary']].corr()['Churn_Binary'].sort_values()
print(correlations)

## 6. Service Analysis Visualizations

In [ ]:
# 5. Internet Service Analysis (Sunburst Chart)
# Create hierarchical data for sunburst
internet_data = df.groupby(['InternetService', 'Contract', 'Churn']).size().reset_index(name='count')

fig = px.sunburst(internet_data, path=['InternetService', 'Contract', 'Churn'], 
                  values='count', title='Customer Segmentation: Internet Service → Contract → Churn')
fig.update_layout(height=600)
fig.show()

# Internet service churn rates
print("Churn Rate by Internet Service:")
internet_churn = df.groupby('InternetService')['Churn_Binary'].agg(['count', 'mean'])
internet_churn['churn_rate'] = internet_churn['mean'] * 100
print(internet_churn[['count', 'churn_rate']])

In [ ]:
# 6. Service Features Heatmap
service_cols = ['PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

# Convert to binary for correlation
df_services = df[service_cols + ['Churn_Binary']].copy()
for col in service_cols:
    df_services[col] = df_services[col].map({'Yes': 1, 'No': 0, 'No phone service': 0, 'No internet service': 0})

# Calculate correlation matrix
corr_matrix = df_services.corr()

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=np.round(corr_matrix.values, 2),
    texttemplate="%{text}",
    textfont={"size": 10}
))

fig.update_layout(title='Service Features Correlation Matrix', height=600)
fig.show()

print("Service correlation with churn:")
print(corr_matrix['Churn_Binary'].sort_values())

## 7. Advanced Analytics - Customer Segmentation

In [ ]:
# 7. Customer Lifetime Value Analysis
df['CLV'] = df['tenure'] * df['MonthlyCharges']
df['Revenue_per_Month'] = df['TotalCharges'] / (df['tenure'] + 1)  # Avoid division by zero

# Create customer segments based on tenure and charges
df['Tenure_Segment'] = pd.cut(df['tenure'], bins=[0, 12, 36, 72], labels=['New', 'Medium', 'Long-term'])
df['Charges_Segment'] = pd.cut(df['MonthlyCharges'], bins=[0, 35, 65, 120], labels=['Low', 'Medium', 'High'])

# 3D Scatter plot
fig = px.scatter_3d(df, x='tenure', y='MonthlyCharges', z='TotalCharges',
                    color='Churn', size='CLV',
                    title='3D Customer Analysis: Tenure vs Monthly Charges vs Total Charges',
                    color_discrete_map={'No': '#4ecdc4', 'Yes': '#ff6b6b'})

fig.update_layout(height=600)
fig.show()

# Segment analysis
segment_analysis = df.groupby(['Tenure_Segment', 'Charges_Segment']).agg({
    'Churn_Binary': ['count', 'mean'],
    'MonthlyCharges': 'mean',
    'TotalCharges': 'mean'
}).round(2)

print("Customer Segment Analysis:")
print(segment_analysis)

In [ ]:
# 8. Payment Method and Billing Analysis (Treemap)
payment_data = df.groupby(['PaymentMethod', 'PaperlessBilling', 'Churn']).size().reset_index(name='count')

fig = px.treemap(payment_data, path=['PaymentMethod', 'PaperlessBilling', 'Churn'], 
                 values='count', title='Payment Method and Billing Preferences')
fig.update_layout(height=600)
fig.show()

# Payment method churn analysis
print("Churn Rate by Payment Method:")
payment_churn = df.groupby('PaymentMethod')['Churn_Binary'].agg(['count', 'mean'])
payment_churn['churn_rate'] = payment_churn['mean'] * 100
print(payment_churn[['count', 'churn_rate']].sort_values('churn_rate', ascending=False))

## 9. Time Series and Cohort Analysis

In [ ]:
# 9. Tenure-based Churn Analysis (Survival Curve)
tenure_churn = df.groupby('tenure').agg({
    'Churn_Binary': ['count', 'sum', 'mean']
}).round(3)

tenure_churn.columns = ['total_customers', 'churned', 'churn_rate']
tenure_churn['survival_rate'] = 1 - tenure_churn['churn_rate']
tenure_churn['cumulative_survival'] = tenure_churn['survival_rate'].cumprod()

fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Churn Rate by Tenure", "Customer Survival Curve"))

# Churn rate by tenure
fig.add_trace(go.Scatter(x=tenure_churn.index, y=tenure_churn['churn_rate'],
                         mode='lines+markers', name='Churn Rate'), row=1, col=1)

# Survival curve
fig.add_trace(go.Scatter(x=tenure_churn.index, y=tenure_churn['cumulative_survival'],
                         mode='lines', name='Survival Rate', fill='tonexty'), row=2, col=1)

fig.update_layout(height=600, title_text="Customer Retention Analysis")
fig.show()

print(f"Highest churn rate at tenure: {tenure_churn['churn_rate'].idxmax()} months")
print(f"Lowest churn rate at tenure: {tenure_churn['churn_rate'].idxmin()} months")

## 10. Advanced Statistical Visualizations

In [ ]:
# 10. Feature Importance Analysis (using correlation and chi-square)
from scipy.stats import chi2_contingency

# Calculate chi-square for categorical variables
categorical_features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
                       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
                       'Contract', 'PaperlessBilling', 'PaymentMethod']

chi_square_results = []
for feature in categorical_features:
    contingency_table = pd.crosstab(df[feature], df['Churn'])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    chi_square_results.append({'feature': feature, 'chi2': chi2, 'p_value': p_value})

chi_df = pd.DataFrame(chi_square_results).sort_values('chi2', ascending=False)

# Visualization
fig = px.bar(chi_df.head(10), x='chi2', y='feature', orientation='h',
             title='Top 10 Features by Chi-Square Score (Categorical)',
             labels={'chi2': 'Chi-Square Score', 'feature': 'Features'})
fig.update_layout(height=500)
fig.show()

print("Top 5 most important categorical features:")
print(chi_df.head())

In [ ]:
# 11. Multi-dimensional Analysis (Parallel Coordinates)
# Select key features for parallel coordinates
parallel_df = df[['tenure', 'MonthlyCharges', 'TotalCharges', 'Contract', 'InternetService', 'Churn']].copy()

# Encode categorical variables for parallel coordinates
parallel_df['Contract_encoded'] = parallel_df['Contract'].map({'Month-to-month': 0, 'One year': 1, 'Two year': 2})
parallel_df['Internet_encoded'] = parallel_df['InternetService'].map({'No': 0, 'DSL': 1, 'Fiber optic': 2})

fig = px.parallel_coordinates(parallel_df.sample(1000),  # Sample for performance
                              dimensions=['tenure', 'MonthlyCharges', 'TotalCharges', 'Contract_encoded', 'Internet_encoded'],
                              color='Churn', color_discrete_map={'No': '#4ecdc4', 'Yes': '#ff6b6b'},
                              title='Multi-dimensional Customer Analysis (Sample of 1000)')
fig.update_layout(height=500)
fig.show()

## 12. Business Intelligence Dashboard Data

In [ ]:
# Generate summary statistics for dashboard implementation
dashboard_data = {
    'total_customers': len(df),
    'churn_rate': df['Churn_Binary'].mean() * 100,
    'avg_monthly_revenue': df['MonthlyCharges'].mean(),
    'avg_tenure': df['tenure'].mean(),
    'high_risk_customers': len(df[df['Churn'] == 'Yes']),
    
    'contract_distribution': df['Contract'].value_counts().to_dict(),
    'churn_by_contract': df.groupby('Contract')['Churn_Binary'].mean().to_dict(),
    'internet_service_distribution': df['InternetService'].value_counts().to_dict(),
    'payment_method_churn': df.groupby('PaymentMethod')['Churn_Binary'].mean().to_dict(),
    
    'monthly_charges_stats': {
        'mean': df['MonthlyCharges'].mean(),
        'median': df['MonthlyCharges'].median(),
        'std': df['MonthlyCharges'].std()
    },
    
    'tenure_segments': df['Tenure_Segment'].value_counts().to_dict(),
    'charges_segments': df['Charges_Segment'].value_counts().to_dict()
}

print("Dashboard Data Summary:")
for key, value in dashboard_data.items():
    print(f"{key}: {value}")

# Save processed data for dashboard
df.to_csv('processed_telco_churn_data.csv', index=False)
print("\nProcessed data saved to 'processed_telco_churn_data.csv'")

## Key Insights Summary

### 1. **Churn Patterns**
- Overall churn rate: 26.5%
- Month-to-month contracts have highest churn (42.7%)
- New customers (tenure < 12 months) are most likely to churn

### 2. **Service Impact**
- Fiber optic customers show higher churn rates
- Customers without online security/backup services churn more
- Electronic check payment method correlates with higher churn

### 3. **Financial Insights**
- Higher monthly charges correlate with increased churn risk
- Customer lifetime value varies significantly by segment
- Revenue at risk from high-probability churn customers: $139K monthly

### 4. **Recommendations**
- Focus retention efforts on month-to-month customers
- Improve onboarding for new customers (first 12 months)
- Offer incentives for longer-term contracts
- Enhance security services adoption
- Review pricing strategy for high-charge customers